In [ ]:
! pip install datasets evaluate transformers diffusers accelerate ftfy pyarrow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.8 MB/s eta 0:00:00


## Tokenizer

In [1]:
from transformers import AutoTokenizer # class

# tokenizers are specific to the models!
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
# distilbert/distilbert-base-uncased-finetuned-sst-2-english
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/Users/chandru/python-sandbox/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_inputs = [
    "I am learning Operating Systems and it's so fun.", # one num
    "I write terrible C++ code!", # sec num
] # text inputs that the model cannot ingest directly
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt", max_length=15)
print(inputs)

{'input_ids': tensor([[ 101, 1045, 2572, 4083, 4082, 3001, 1998, 2009, 1005, 1055, 2061, 4569,
         1012,  102],
        [ 101, 1045, 4339, 6659, 1039, 1009, 1009, 3642,  999,  102,    0,    0,
            0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])}


In [ ]:
[4, 5, 6] # row 1 -> inp 1
[1, 2, <pad_tokens>] # row 2 -> inp 2

# unable to form a matrix -> matmul not possible -> model processing np

In [4]:
# another way to do it - a closer look into the tokenizer
sequence = "I am learning Operating Systems and it's so fun."
tokens = tokenizer.tokenize(sequence)

print("--------TOKENIZED SENTENCE--------")
print(tokens)

print("\n\n--------TOKENS MAPPED TO THEIR IDS--------")
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

--------TOKENIZED SENTENCE--------
['i', 'am', 'learning', 'operating', 'systems', 'and', 'it', "'", 's', 'so', 'fun', '.']


--------TOKENS MAPPED TO THEIR IDS--------
[1045, 2572, 4083, 4082, 3001, 1998, 2009, 1005, 1055, 2061, 4569, 1012]


**TAKE A PAUSE!**

**SPECIAL TOKENS!**

In [5]:
# already tokenized input

# mapping the token ids back to the tokens
decoded_string = tokenizer.decode([1045, 2572, 4083, 4082, 3001, 1998, 2009, 1005, 1055, 2061, 4569, 1012])
print(decoded_string)

i am learning operating systems and it ' s so fun.


In [6]:
# raw text
decoded_string = tokenizer.decode([101, 1045, 2572, 4083, 4082, 3001, 1998, 2009, 1005, 1055, 2061, 4569,
         1012,  102])
print(decoded_string)

[CLS] i am learning operating systems and it ' s so fun. [SEP]


**Padding - ways to do it**

In [9]:
# Will pad the sequences up to the maximum sequence length
model_inputs = tokenizer(sequence, padding="longest")

print("--------TOKEN IDS (after pad the sequences up to the maximum sequence length)--------")
print(model_inputs["input_ids"])

print("\n--------DECODED STRING (after pad the sequences up to the maximum sequence length)--------")
decoded_padded_string = tokenizer.decode(model_inputs["input_ids"])
print(decoded_padded_string)



# Will pad the sequences up to the model max length
# (512 for BERT or DistilBERT) model max length = the max sentence length that the model can ingest
model_inputs = tokenizer(sequence, padding="max_length")



print("--------TOKEN IDS (after Will pad the sequences up to the model max length)--------")
print(model_inputs["input_ids"])

print("\n--------DECODED STRING (after Will pad the sequences up to the model max length)--------")
decoded_padded_string = tokenizer.decode(model_inputs["input_ids"])
print(decoded_padded_string)



# Will pad the sequences up to the specified max length
model_inputs = tokenizer(sequence, padding="max_length", max_length=8)

print("--------TOKEN IDS (after padding to max_length=8)--------")
print(model_inputs["input_ids"])

print("\n--------DECODED STRING (after padding to max_length=8)--------")
decoded_padded_string = tokenizer.decode(model_inputs["input_ids"])
print(decoded_padded_string)

--------TOKEN IDS (after pad the sequences up to the maximum sequence length)--------
[101, 1045, 2572, 4083, 4082, 3001, 1998, 2009, 1005, 1055, 2061, 4569, 1012, 102]

--------DECODED STRING (after pad the sequences up to the maximum sequence length)--------
[CLS] i am learning operating systems and it ' s so fun. [SEP]
--------TOKEN IDS (after Will pad the sequences up to the model max length)--------
[101, 1045, 2572, 4083, 4082, 3001, 1998, 2009, 1005, 1055, 2061, 4569, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Models

In [10]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

Loading weights: 100%|██████████| 100/100 [00:00<00:00, 2616.22it/s, Materializing param=transformer.layer.5.sa_layer_norm.weight]   
DistilBertModel LOAD REPORT from: distilbert-base-uncased-finetuned-sst-2-english
Key                   | Status     |  | 
----------------------+------------+--+-
pre_classifier.weight | UNEXPECTED |  | 
classifier.weight     | UNEXPECTED |  | 
pre_classifier.bias   | UNEXPECTED |  | 
classifier.bias       | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [11]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape) # bs, seq_len, dim

torch.Size([2, 14, 768])


In [12]:
from transformers import AutoModelForSequenceClassification
import torch

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)


# there are a total of 2 classes -- use this info to make sense of the shape of logits
print("--------SHAPE OF LOGIT TENSOR--------")
print(outputs.logits.shape)

print("\n\n--------LOGIT TENSOR--------")
print(outputs.logits)

# convert logits to probs --> softmax (math func)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = torch.argmax(predictions, dim=1)
print("\n\n--------PREDICTIONS--------")
print(predictions) # os one, c++ one

Loading weights: 100%|██████████| 104/104 [00:00<00:00, 2759.64it/s, Materializing param=pre_classifier.weight]                                  

--------SHAPE OF LOGIT TENSOR--------
torch.Size([2, 2])


--------LOGIT TENSOR--------
tensor([[-4.3357,  4.6875],
        [ 4.6717, -3.7884]], grad_fn=<AddmmBackward0>)


--------PREDICTIONS--------
tensor([1, 0])


In [13]:
# homework: what is 1? negative or positive?
# figure out the mapping b/w numbers and their corresponding labels
# else post on circle to receive mentor help

# Print the mapping
print(model.config.id2label)

# To answer your specific question:
label_for_1 = model.config.id2label[1]
print(f"The label for '1' is: {label_for_1}")



{0: 'NEGATIVE', 1: 'POSITIVE'}
The label for '1' is: POSITIVE
